In [1]:
import numpy as np


def concatenate(X):
    return np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)


def error(X, y, model):
    return model.predict_proba(X)[:, 1] - y


def prediction(X, model):
    return model.predict_proba(X)[:, 1]


def gradient_(X, y, model, C):
    F = concatenate(X)
    #return F * error(X, y, model)[:, None] + C * w / X.shape[0]
    return F * error(X, y, model)[:, None]


def hessian_(X, model, C):
    probs = prediction(X, model)
    F = concatenate(X)
    H = F.T @ np.diag(probs * (1 - probs)) @ F / F.shape[0] + C * np.eye(
        F.shape[1]) / X.shape[0]
    return H


def inverse_hessian(H):
    return np.linalg.inv(H)

In [2]:
def Remove(k, scores, test_idx):
    #print("test_idx", test_idx)
    #print("old")
    #print(pred[test_idx])
    
    if pred[test_idx] > 0.5:
        top_k_index = scores[test_idx].argsort()[-k:]
    else:
        top_k_index = scores[test_idx].argsort()[:k]
    
    X_r = X["train"][top_k_index]
    y_r = y["train"][top_k_index]
    X_l = np.delete(X["train"], top_k_index, axis=0)
    y_l = np.delete(y["train"], top_k_index, axis=0)

        
    prediction = -np.sum(scores[test_idx][top_k_index])
    #print("prediction", prediction)

    return X_l, y_l, prediction, X_r, y_r



In [4]:
def new_train(k, dev_index, scores):
    X_k, y_k, prediction, x_r, y_r = Remove(k, scores, dev_index)
    
    if y_k.shape[0] == np.sum(y_k) or np.sum(y_k) == 0: # data contains only one class: 1
        return None

    # Fit the model again
    model_k = LogisticRegression(penalty='l2', C=0.1)
    model_k.fit(X_k, y_k)

    # predictthe probaility with test point
    test_point = X["dev"][dev_index]
    test_point=np.reshape(test_point, (1,-1))
    
    new = model_k.predict_proba(test_point)[0][1]
    return new

In [5]:
def approximate_k(test_idx):
    old = pred[test_idx].item()
    for k in range(1, np.sum(y["train"])):
        X_l, y_l, pred_change_if, X_r, y_r = Remove(k, delta_pred, test_idx)
        #_, flip,_ = new_train(k, test_idx, delta_pred)
        
        if old > 0.5 and pred_change_if+old <0.5:
            return k
        elif old < 0.5 and pred_change_if+old > 0.5:
            return k
        
    return None

In [6]:
def recursive_NT(test_idx, old, X_0, y_0, model, l2, I=100, D=1):
    eps = 1 / X["train"].shape[0]
    X_l = X_0
    y_l = y_0
    X_r = X_0
    y_r = y_0
    # Compute IP
    F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)
    new_hessain = hessian_(X_l, model, l2)
    new_inv = inverse_hessian(new_hessain)
    new_grad_train = gradient_(X_r, y_r, model, l2)
    delta_k = -eps * new_inv @ new_grad_train.T
    grad_f = F_dev[test_idx] * (old * (1 - old))
    delta_pred = grad_f @ delta_k

    K_new = y_0.shape[0]
    predicted_change_new = None
    ite = 0
    diff = K_new - 0
    removed_order = []
    while (ite < I and diff > D and K_new != 1):
        ite += 1

        if old > 0.5:
            sorted_index = np.flip(delta_pred.argsort())
        else:
            sorted_index = delta_pred.argsort()

        for k in range(1, y_r.shape[0]):
            top_k_index = sorted_index[:k]
            predicted_change = -np.sum(delta_pred[top_k_index])
            """
            print("K", k)
            print("predicted_change", predicted_change)
            print("old", np.round(old))
            print("K", k)
            print("removed shape", X_r.shape)
            print("left shape", X_l.shape)
            print("delta_pred shape", delta_pred.shape)
            print("predicted_change", predicted_change)
            print("old", np.round(old), "new", np.round(old + predicted_change))
            print()
            """

            if np.round(old) != np.round(old + predicted_change):
                # print("K", k)

                diff = K_new - k
                K_new = k
                predicted_change_new = predicted_change

                index_whole = []
                for idx_r in top_k_index:
                    point = X_r[idx_r]
                    idx_0 = X_dist[str(point.tolist())]
                    index_whole.append(idx_0)

                X_r = X_r[top_k_index]
                y_r = y_r[top_k_index]

                X_l = np.delete(X_0, index_whole, axis=0)
                y_l = np.delete(y_0, index_whole, axis=0)
                # print("removed shape", X_r.shape)
                # print("left shape", X_l.shape)

                # new hessian for the left points
                new_hessain = hessian_(X_l, model, C=l2)
                new_inv = inverse_hessian(new_hessain)
                # new gradient for the removed points
                new_grad_train = gradient_(X_r, y_r, model, C=l2)

                delta_k = -eps * new_inv @ new_grad_train.T
                grad_f = F_dev[test_idx] * (old * (1 - old))
                delta_pred = grad_f @ delta_k
                break

            if k == y_r.shape[0] - 1:
                if K_new == y_0.shape[0] and diff == y_0.shape[0]:
                    return None, predicted_change_new, ite, None, []
                
                return K_new, predicted_change_new, ite, diff, index_whole

    return K_new, predicted_change_new, ite, diff, index_whole

In [8]:
import numpy as np
import warnings
X = {}
y = {}
X["train"] = np.load("X_train.npy")
y["train"] = np.load("y_train.npy")
X["dev"] = np.load("X_dev.npy")
y["dev"] = np.load("y_dev.npy")
X["dev"].shape, y["dev"].shape

((1003, 300), (1003,))

In [8]:
from sklearn.linear_model import LogisticRegression
l2 = 10
model = LogisticRegression(penalty='l2', C=1/l2)
model.fit(X["train"], y["train"])
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
model.score(X["dev"], y["dev"])


0.7666999002991027

In [9]:
w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))], axis=1) # Concatenating one to calculate the gradient with respect to intercept
F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

gradient_train = F_train * error_train[:, None]
gradient_dev = F_dev * error_dev[:, None]
#from scipy import sparse
probs = model.predict_proba(X["train"])[:, 1]
H = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + l2 * np.eye(F_train.shape[1]) / X["train"].shape[0]
inv_H = np.linalg.inv(H)

eps = 1 / X["train"].shape[0]
delta_k = -eps * inv_H @ gradient_train.T
grad_f = F_dev * (pred * (1 - pred))
delta_pred = grad_f @ delta_k



In [ ]:
X_dist = {}
for i in range(X["train"].shape[0]):
    X_dist[str(X["train"][i].tolist())] = i
    
NT_app_k = []
new_predictions = []
iterations = []
diffs = []
order_lists = []

for i in range(X["dev"].shape[0]):
    test_idx = i
    print("test_idx", test_idx)

    old = pred[test_idx].item()
    print("old", old)

    K_nt, pred_change_nt, ite, diff, order= recursive_NT(test_idx, old, X["train"], y["train"], model, l2, I=100, D=1)

    if pred_change_nt != None:  
        new_nt = new_train(K_nt, test_idx, delta_pred)
    else:
        new_nt = None

        
    print("K_nt, pred_change_nt, ite, diff")
    print(K_nt, pred_change_nt, ite, diff)
    print("new", new_nt)
    print("order", len(order))
    print()
    NT_app_k.append(K_nt)
    new_predictions.append(new_nt)
    iterations.append(ite)
    diffs.append(diff)
    order_lists.append(order)

np.save("NT_app_k_emo_LR_I100_D1.npy", NT_app_k)
np.save("new_predictions_emo_LR_I100_D1.npy", new_predictions)
np.save("iterations_emo_LR_I100_D1.npy", iterations)
np.save("diffs_emo_LR_I100_D1.npy", diffs)
np.save("order_emo_LR_I100_D1.npy", order_lists)

test_idx 0
old 0.4281949532461735
K_nt, pred_change_nt, ite, diff
38 0.07233874209289835 3 5
new 0.5019768601876896
order 38

test_idx 1
old 0.6384329207747343
K_nt, pred_change_nt, ite, diff
118 -0.13890902110536002 3 21
new 0.4901647797597792
order 118

test_idx 2
old 0.13458845458938548
K_nt, pred_change_nt, ite, diff
1681 0.3655144732019987 5 1
new 0.8567202318442897
order 1681

test_idx 3
old 0.24096942964916382
K_nt, pred_change_nt, ite, diff
214 0.25937846460084824 3 73
new 0.5663368914249876
order 214

test_idx 4
old 0.6855117940043363
K_nt, pred_change_nt, ite, diff
130 -0.18556294384964536 3 38
new 0.48050840693925984
order 130

test_idx 5
old 0.4189420896037263
K_nt, pred_change_nt, ite, diff
94 0.08112837307369984 3 9
new 0.5026410541681466
order 94

test_idx 6
old 0.11920936188965771
K_nt, pred_change_nt, ite, diff
1144 0.38093829531410495 4 63
new 0.8880149777787522
order 1144

test_idx 7
old 0.5274833241784838
K_nt, pred_change_nt, ite, diff
14 -0.027520758098833514 2 1


/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


K_nt, pred_change_nt, ite, diff
154 0.08029769776458769 3 16
new 0.5030487040330164
order 154

test_idx 65
old 0.866447635412085
K_nt, pred_change_nt, ite, diff
1453 -0.3665464922644204 4 192
new 0.1441641530920525
order 1453

test_idx 66
old 0.625505669384155
K_nt, pred_change_nt, ite, diff
96 -0.1255158330603018 3 14
new 0.4932412679148104
order 96

test_idx 67
old 0.7155211269659718
K_nt, pred_change_nt, ite, diff
283 -0.21596458623775217 3 82
new 0.4575181647311809
order 283

test_idx 68
old 0.22264488606651306
K_nt, pred_change_nt, ite, diff
240 0.2776468454948039 3 65
new 0.5927036476695742
order 240

test_idx 69
old 0.3371979473140052
K_nt, pred_change_nt, ite, diff
253 0.16307396432940327 3 55
new 0.5212061198958579
order 253

test_idx 70
old 0.7159115772065943
K_nt, pred_change_nt, ite, diff
347 -0.21609371155346696 3 95
new 0.45567414166981524
order 347

test_idx 71
old 0.5725944334280839
K_nt, pred_change_nt, ite, diff
92 -0.0726157020779249 3 11
new 0.4979480625539183
order

K_nt, pred_change_nt, ite, diff
85 -0.10882115316078314 3 12
new 0.49493680433026155
order 85

test_idx 129
old 0.8176496940338585
K_nt, pred_change_nt, ite, diff
816 -0.317661269755304 3 359
new 0.30362687187094384
order 816

test_idx 130
old 0.648733764461008
K_nt, pred_change_nt, ite, diff
179 -0.14903198755818373 3 34
new 0.4868970274640872
order 179

test_idx 131
old 0.1245718221096413
K_nt, pred_change_nt, ite, diff
720 0.3755798690701745 4 7
new 0.8785025286600905
order 720

test_idx 132
old 0.2416038139498617
K_nt, pred_change_nt, ite, diff
284 0.25845670342340576 3 96
new 0.5756354343819592
order 284

test_idx 133
old 0.812919570997703
K_nt, pred_change_nt, ite, diff
1157 -0.31294713198503693 4 12
new 0.2936832081373509
order 1157

test_idx 134
old 0.9047726046335735
K_nt, pred_change_nt, ite, diff
None None 1 None
new None
order 0

test_idx 135
old 0.6845063613152873
K_nt, pred_change_nt, ite, diff
229 -0.18491802984520045 3 56
new 0.4735639515851585
order 229

test_idx 136
o

K_nt, pred_change_nt, ite, diff
None None 1 None
new None
order 0

test_idx 193
old 0.7333467127484448
K_nt, pred_change_nt, ite, diff
233 -0.23390351005638355 3 57
new 0.4393443622221638
order 233

test_idx 194
old 0.17762054538418956
K_nt, pred_change_nt, ite, diff
671 0.32252752997974765 3 282
new 0.707608751999931
order 671

test_idx 195
old 0.47421067933210376
K_nt, pred_change_nt, ite, diff
11 0.025939692431982506 2 9014
new 0.501117689665994
order 11

test_idx 196
old 0.45102737096510986
K_nt, pred_change_nt, ite, diff
34 0.04954557618763303 3 3
new 0.5010292583847096
order 34

test_idx 197
old 0.5564560225130535
K_nt, pred_change_nt, ite, diff
47 -0.05647282635375844 3 5
new 0.4993481049990178
order 47

test_idx 198
old 0.5810720907616387
K_nt, pred_change_nt, ite, diff
74 -0.0812703200613367 3 14
new 0.49777809200728323
order 74

test_idx 199
old 0.4166575638354492
K_nt, pred_change_nt, ite, diff
61 0.08381153368121029 3 6
new 0.503746863274871
order 61

test_idx 200
old 0.338

# Bert lin

In [29]:
X = {}
y = {}
X["train"] = np.load("train_feature_save.npy")[:9025]
y["train"] = np.load("train_label_save.npy").squeeze()[:9025]
X["dev"] = np.load("test_feature_save.npy")[:1003]
y["dev"] = np.load("test_label_save.npy").squeeze()[:1003]


# In[14]:


X["train"].shape, X["dev"].shape, y["train"].shape, y["dev"].shape




((9025, 768), (1003, 768), (9025,), (1003,))

In [33]:
# In[15]:
"""

l2 = 10
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=1/l2)
model.fit(X["train"], y["train"])
model.score(X["dev"], y["dev"])
w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))], axis=1) # Concatenating one to calculate the gradient with respect to intercept
F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

gradient_train = F_train * error_train[:, None]
gradient_dev = F_dev * error_dev[:, None]

#from scipy import sparse
probs = model.predict_proba(X["train"])[:, 1]
H = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + l2 * np.eye(F_train.shape[1]) / X["train"].shape[0]
inverse_H = np.linalg.inv(H)

eps = 1 / X["train"].shape[0]
delta_k = -eps * inverse_H @ gradient_train.T
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
grad_f = F_dev * (pred * (1 - pred))
delta_pred = grad_f @ delta_k
delta_pred.shape


# In[18]:


X_dist = {}
for i in range(X["train"].shape[0]):
    X_dist[str(X["train"][i].tolist())] = i

NT_app_k = []
new_predictions = []
iterations = []
diffs = []
order_lists = []

for i in range(X["dev"].shape[0]):
    test_idx = i
    print("test_idx", test_idx)

    old = pred[test_idx].item()
    print("old", old)

    K_nt, pred_change_nt, ite, diff, order= recursive_NT(test_idx, old, X["train"], y["train"], model, l2, I=100, D=1)

    if pred_change_nt != None:  
        new_nt = new_train(K_nt, test_idx, delta_pred)
    else:
        new_nt = None

        
    print("K_nt, pred_change_nt, ite, diff")
    print(K_nt, pred_change_nt, ite, diff)
    print("new", new_nt)
    print()
    NT_app_k.append(K_nt)
    new_predictions.append(new_nt)
    iterations.append(ite)
    diffs.append(diff)
    order_lists.append(order)

np.save("NT_app_k_emo_bertT_I100_D1.npy", NT_app_k)
np.save("new_predictions_emo_bertT_I100_D1.npy", new_predictions)
np.save("iterations_emo_bertT_I100_D1.npy", iterations)
np.save("diffs_emo_bertT_I100_D1.npy", diffs)
np.save("order_emo_bertT_I100_D1.npy", order_lists)
"""

/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


test_idx 0
old 0.5608479913294804


/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


K_nt, pred_change_nt, ite, diff
18 -0.06253433387828902 2 1
new 0.385435930631118

test_idx 1
old 0.8977539867734747


/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


K_nt, pred_change_nt, ite, diff
1354 -0.41639235628024024 5 4
new 2.9576465942843647e-05

test_idx 1000
old 0.9370985528258465
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 1001
old 0.6512689697772442


/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


K_nt, pred_change_nt, ite, diff
72 -0.1522922234610368 3 11
new 0.2181378828589316

test_idx 1002
old 0.7113038966855577
K_nt, pred_change_nt, ite, diff
106 -0.2114257237888627 3 41
new 0.4049034652049578



/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/envs/torch/lib/python3.8/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
